In [1]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

#TODO: subtract outgoing

import urllib.request, json
import csv
import itertools
import math
import collections
import sys

import grequests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

sys.path.insert(0, '../')
import sc_pull

%load_ext autoreload
%autoreload 1
pd.set_option('display.max_columns', 500)

In [32]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

#TODO: subtract outgoing

import urllib.request, json
import csv
import itertools
import math
import collections
import sys

import grequests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize


class fts(object):
    def __init__(self, sc, test=None, all_sector=False):
        self.all_sector = all_sector
        self.test = test
        self.sc = sc

        self.fts = self.master_pull()
        self.merge_sc()
        self.  = self.extract_flows()

        # remove flows col as it's quite large
        self.fts = self.fts.drop('fts.data.flows', axis=1)

    def extract_flows(self):
        """take all flows and add to df"""

        flow_df = pd.DataFrame()
        for v in self.fts.iterrows():
            norm = json_normalize(v[1]['fts.data.flows'])
            norm['fts.uid'] = v[1]['sc.uid']

            if len(flow_df) == 0:
                flow_df = norm
            else:
                # need to make sure we're adding t3h same c0lz
                assert (norm.columns.all(flow_df.columns))
                flow_df = flow_df.append(norm, ignore_index=True)

        return flow_df

    def master_pull(self):
        """group by year, country and pull down total values with optional filter by cluster"""
        countries = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/location")
                                              .read().decode()))['data']

        print('pulled countries')

        self.hist = self.sc

        def get_other_sects():
            """get comma seperated string of all other sectors if needed"""
            r = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/global-cluster")
                                          .read().decode()))['data']

            r = [v['code'] for v in r[0]]
            r.remove('SHL')
            return ','.join(r)

        self.hist['sc.fts_url'] = self.hist.apply(lambda x: \
                          'https://api.hpc.tools/v1/public/fts/flow?countryISO3={0}&year={1}' \
                          '&globalClusterCode={2}'.format(x['sc.iso3'], x['sc.year'], \
                                                          'SHL' if not self.all_sector else get_other_sects()), axis=1)

        if self.test:
            hrefz = self.hist['sc.fts_url'][:self.test]
        else:
            hrefz = self.hist['sc.fts_url']

        def exception_handler(request, exception):
            print('Bad URL for ' + request)

        resps = []
        rs = (grequests.get(ref) for ref in hrefz)
        resps += grequests.map(rs, exception_handler=exception_handler, size=25)

        good_resps = []
        bad_resps = []
        for r in resps:
            load = json.loads(r.content)
            load['url'] = r.url
            if r.status_code == 200:
                good_resps.append(load)
            else:
                bad_resps.append(load)

        print('num bad resps: ' + str(len(bad_resps)))
        return json_normalize(good_resps).add_prefix('fts.')

    def merge_sc(self):
        self.fts = self.fts.merge(self.hist, left_on='fts.url', right_on='sc.fts_url')

    def get_flow_bdown(self):
        """break down flows by needed columns"""

        def hp(v):
            if 'Plan' in json_normalize(v['destinationObjects']).values or 'Plan' in json_normalize(
                    v['sourceObjects']).values:
                return 'has_plan'
            else:
                return 'no_plan'

        self.flows['plan'] = self.flows.apply(lambda x: hp(x), axis=1)

        # we are skipping boundary for now as it's <<<<
        d = self.flows[['fts.uid', 'plan', 'amountUSD']].groupby(['fts.uid', 'plan'], as_index=False).aggregate(
            {'amountUSD': 'sum'})
        piv = pd.pivot_table(d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[d['plan']] \
                             , fill_value=0).reset_index()
        piv['total'] = piv['has_plan'] + piv['no_plan']
        return piv


import sc_pull
sc = sc_pull.pull()
# s_f = fts(test=10, sc=sc)
all_f = fts(test=1, sc=sc, all_sector=True)

# print(s_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 sc
print(all_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 all
#
# # AGR,CCM,PRO-CPN,CSS,ERY,EDU,TEL,FSC,PRO-GBV,HEA,PRO-HLP,LOG,PRO-MIN,MS,NUT,OTH,PRO,WSH


loaded 227 SC entries
pulled countries
num bad resps: 0
plan sc_fts.uid  sc_has_plan  sc_no_plan   sc_total
0       PAK2005     57609250   140214092  197823342


In [34]:
all_f.fts

,fts.data.incoming.flowCount,fts.data.incoming.fundingTotal,fts.data.incoming.pledgeTotal,fts.data.internal.flowCount,fts.data.outgoing.flowCount,fts.data.outgoing.fundingTotal,fts.data.outgoing.pledgeTotal,fts.meta.count,fts.meta.language,fts.meta.nextLink,fts.status,fts.url,sc.region,sc.region_url,sc.country,sc.country_url,sc.year,sc.response,sc.lead_agency,sc.disaster_type,sc.website_url,sc.colead_agency,sc.activation_date,sc.deactivation_date,sc.num_clust,sc.iso3,sc.uid,sc.fts_url
0,770,872497705,96172701,0,0,0,0,770,en,https://api.hpc.tools/v1/public/fts/flow?count...,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Pakistan,http://sheltercluster.org/asiapacific/pakistan,2005,Pakistan - Kashmir Earthquake,IOM,Natural Disaster,NaN,NaN,NaN,NaN,1,PAK,PAK2005,https://api.hpc.tools/v1/public/fts/flow?count...


In [13]:
sf_b

plan,sc_fts.uid,sc_has_plan,sc_no_plan,sc_total
0,AFG2008,277830369,273619919,551450288
1,AFG2009,500923100,135283677,636206777
2,AFG2010,445435796,132288105,577723901
3,AFG2011,319833752,294657607,614491359
4,AFG2012,218512650,245684360,464197010
5,AFG2013,247333370,86925979,334259349
6,AFG2014,435565007,87616854,523181861
7,AFG2015,421989011,80103470,502092481
8,AFG2016,239127406,129954143,369081549
9,AFG2017,298242850,57013427,355256277


In [19]:
sf_b

plan,sc_fts.uid,sc_has_plan,sc_no_plan,sc_total
0,BGD2007,0,13419063,13419063
1,CAF2007,1158174,589370,1747544
2,COD2006,6172648,2303036,8475684
3,IDN2006,16111160,16722196,32833356
4,LBN2006,23960141,61236610,85196751
5,LBR2006,0,341267,341267
6,PAK2005,21137104,94081423,115218527
7,PHL2006,379586,2918874,3298460
8,SOM2006,3469747,1243286,4713033
9,UGA2006,1496344,2316002,3812346


In [21]:
mrgd

plan,sc_fts.uid,sc_has_plan,sc_no_plan,sc_total,non_sc_fts.uid,non_sc_total
0,BGD2007,0,13419063,13419063,BGD2007,295616727
1,CAF2007,1158174,589370,1747544,CAF2007,97317487
2,COD2006,6172648,2303036,8475684,COD2006,185909603
3,IDN2006,16111160,16722196,32833356,IDN2006,79337369
4,LBN2006,23960141,61236610,85196751,LBN2006,185195836
5,LBR2006,0,341267,341267,LBR2006,123814325
6,PAK2005,21137104,94081423,115218527,PAK2005,197823342
7,PHL2006,379586,2918874,3298460,PHL2006,28340367
8,SOM2006,3469747,1243286,4713033,SOM2006,236399871
9,UGA2006,1496344,2316002,3812346,UGA2006,248035455


In [31]:
import sc_pull
sc = sc_pull.pull()
all_f = fts(test = 10, sc = sc, all_sector=True)
s_f = fts(test = 10, sc = sc)

all_b = all_f.get_flow_bdown().add_prefix('non_sc_')
sf_b = s_f.get_flow_bdown().add_prefix('sc_')

mrgd = sf_b.merge(all_b[['non_sc_fts.uid', 'non_sc_total']], left_on='sc_fts.uid', right_on='non_sc_fts.uid', how='left')
# mrgd = sf_b.merge(all_b, left_on='sc_fts.uid', right_on='non_sc_fts.uid', how='left')
mrgd.drop('non_sc_fts.uid', axis=1)


loaded 227 SC entries
pulled countries
num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


pulled countries
num bad resps: 0


plan,sc_fts.uid,sc_has_plan,sc_no_plan,sc_total,non_sc_total
0,BGD2007,0,13419063,13419063,295616727
1,CAF2007,1158174,589370,1747544,97317487
2,COD2006,6172648,2303036,8475684,185909603
3,IDN2006,16111160,16722196,32833356,79337369
4,LBN2006,23960141,61236610,85196751,185195836
5,LBR2006,0,341267,341267,123814325
6,PAK2005,21137104,94081423,115218527,197823342
7,PHL2006,379586,2918874,3298460,28340367
8,SOM2006,3469747,1243286,4713033,236399871
9,UGA2006,1496344,2316002,3812346,248035455


In [5]:
all_f.get_flow_bdown()

plan,fts.uid,has_plan,no_plan,total
0,BGD2007,0,302490807,302490807
1,CAF2007,69117871,7857834,76975705
2,COD2006,229043523,48256475,277299998
3,IDN2006,40413306,62957052,103370358
4,LBN2006,78798540,674452264,753250804
5,LBR2006,112841782,45793646,158635428
6,PAK2005,91251841,260261360,351513201
7,PHL2006,7218561,26733251,33951812
8,SOM2006,193842936,47725414,241568350
9,UGA2006,204032801,44671465,248704266


In [31]:
r['fts.url'].values

array(['https://api.hpc.tools/v1/public/fts/flow?countryISO3=PAK&year=2005&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=COD&year=2006&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=IDN&year=2006&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=LBN&year=2006&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=LBR&year=2006&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=PHL&year=2006&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=SOM&year=2006&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=UGA&year=2006&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=BGD&year=2007&globalClusterCode=',
       'https://api.hpc.tools/v1/public/fts/flow?countryISO3=CAF&year=2007&globalClusterCode='],
      dtype=object)

In [46]:
import sc_pull
sc = sc_pull.pull()
s_f = fts(test=10, sc=sc)
# all_f = fts(test=10, sc=sc, all_sector=True)

# # print(s_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 sc
# print(all_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 all


loaded 227 SC entries
pulled countries
num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [68]:
all_b = all_f.get_flow_bdown().add_prefix('non_sc_')
sf_b = s_f.get_flow_bdown().add_prefix('sc_')


In [73]:
#all: 197823342

#sf_b: 115218527 197823342

all_b['y'] = all_b.apply(lambda x : x['non_sc_fts.uid'][3:], axis = 1)

all_b = all_b.sort_values(by = 'y')
all_b.to_csv('../d0cz/all_out.csv')

In [10]:
sf_b

plan,sc_fts.uid,sc_has_plan,sc_no_plan,sc_total
0,AFG2008,277830369,273619919,551450288
1,AFG2009,500923100,135283677,636206777
2,AFG2010,445435796,132288105,577723901
3,AFG2011,319833752,294657607,614491359
4,AFG2012,218512650,245684360,464197010
5,AFG2013,247333370,86925979,334259349
6,AFG2014,435565007,87616854,523181861
7,AFG2015,421989011,80103470,502092481
8,AFG2016,239127406,129954143,369081549
9,AFG2017,298242850,57013427,355256277


In [581]:
#in progress

gran_cols = ['incoming_commitment', 'incoming_paid', 'incoming_pledge', 'outgoing_commitment', 
            'outgoing_paid', 'outgoing_pledge', 'internal_commitment', 'internal_paid', 'internal_pledge']


def check_sum(uid):
    """check sums are good"""
    sm = sum([ft[ft['sc.uid'] == uid][g] for g in gran_cols if ft[ft['sc.uid'] == uid][g]][0])

def pop_flows():
    g = fs.groupby(by = ['sc.uid', 'boundary', 'status']).agg({'amountUSD' : 'sum'})
    g = fs[[‘amountUSD’,‘boundary', 'status’]].groupby(‘boundary', 'status’).sum()
    g = g.reset_index()
        
    for c in gran_cols:
        ft[c] = None

    for v in g.iterrows():
        var_nm = v[1]['boundary'] + '_' + v[1]['status']
        
        if len(ft.loc[ft['sc.uid'] == v[1]['sc.uid']]) != 1:
            raise Exception('bad UID')
        
        if v[1]['sc.uid'] not in list(ft['sc.uid']):
            print(v[1]['sc.uid'])
        
        ft.loc[ft['sc.uid'] == v[1]['sc.uid'], var_nm] = v[1]['amountUSD']
        
        #check all sums are good
#         check_sum(v[1]['sc.uid'])
        
#     df[col_list].sum(axis=1)
    
pop_flows()

SyntaxError: invalid character in identifier (<ipython-input-581-41cc2d97e67c>, line 11)

In [ ]:
bounds = {}
def get_bound_types(x):
    """see what count boundary transactions we have
        {'incoming': 4889, 'internal': 10, 'outgoing': 23}
    """
    norm = json_normalize(x['fts.data.flows'])
    
    if 'boundary' in norm:
        for v in norm['boundary']:
            if v == 'outgoing':
                print(x)
                raise('sfd')

            if v not in bounds:
                bounds[v] = 1
            else:
                bounds[v] += 1
    
j.apply(lambda x : get_bound_types(x), axis = 1)

In [ ]:
#test to see if we're figuring out outgoing correctly
ent = j[j['fts.data.incoming.fundingTotal'] == 1597544]

#regular:
json_normalize(ent['fts.data.flows'][9]).groupby(by = 'status')['amountUSD'].sum()
# commitment    1130923
# paid           616621

#subtract:
r = json_normalize(ent['fts.data.flows'][9]).groupby(by = ['status', 'boundary'])['amountUSD'].sum()

list(set(r.index.get_level_values('status')))

r.keys()